# Clasificador Bayesiano


### Librerías a utilizar:

- e1071
- caret

### ¿Qué es un clasificador? ¿Bayesiano?
Un clasificador corresponde a un modelo de aprendizaje automático que es capaz de reconocer patrones respecto a un valor objetivo. Para esta sesión utilizaremos y estudiaremos el clasificador bayesiano ingenuo, el cual se basa en la probabilidad condicional para "aprender" a clasificar.

Un recordatorio, ¿qué es el teorema de bayes?

> El teorema de Bayes, en la teoría de la probabilidad, es una proposición planteada por el matemático inglés Thomas Bayes (1702-1761)1​ y publicada póstumamente en 1763,2​ que expresa la probabilidad condicional de un evento aleatorio A dado B en términos de la distribución de probabilidad condicional del evento B dado A y la distribución de probabilidad marginal de solo A. (Wikipedia)

$$ p(A|B) = \frac{p(A \cap B)}{p(B)} = \frac{p(A)p(B|A)}{p(B)} $$

Por lo tanto, para crear un clasificador bayesiano solo hace falta conocer las probabilidades condicionales de las caracteristicas en conjunto a la ocurrencia de la clase.

$$ Clase_{NBC} = arg max \{ p(c_i) \Pi^p_i p(a_j / c_i)) \} $$

- $p(c_i)$ = probabilidad a priori
- $p(a_j / c_i)$ = probabilidad a posteriori

Cuando en la probabilidad a posteriori se tienen numeros reales, se calcula:

$$ p(a_j/c_i) = \frac{1}{\sqrt(2 \cdot \pi) \cdot \sigma_j } \cdot e^{\frac{(a_j - \bar{a_j})^2}{2 \cdot \sigma_j}}$$


Si quisieramos evaluar la diabetes Mellitus, en donde supuestamente existen dos variables de interés, el nivel sobrepeso y el nivel de glucosa en la sangre del paciente. De un hospital se tiene la siguiente tabla:

|  			Nivel de sobrepeso 		 |  			Glucosa en mg/100 ml de sangre 		 |  			Diabético 		 |
|:--------------------:|:--------------------------------:|:-----------:|
|        			Normal 		       |                			70 		               |      			No 		    |
|        			Normal 		       |                			80 		               |      			No 		    |
|       			Sobrepeso 		     |                			90 		               |      			No 		    |
|        			Normal 		       |                			100 		              |      			No 		    |
|         			Obeso 		       |                			110 		              |      			No 		    |
|       			Sobrepeso 		     |                			120 		              |      			Sí 		    |
|       			Sobrepeso 		     |                			130 		              |      			Sí 		    |
|         			Obeso 		       |                			140 		              |      			Sí 		    |
|         			Obeso 		       |                			150 		              |      			Sí 		    |



Tenemos que la clase corresponde a Diabetes={si, no}, por lo tanto la probabilidad a priori corresponde a:

- p(Diabetes=si) = 5/9
- p(Diabetes=no) = 4/9

Luego se deben calcular las probabilidades a posteriori:

- p(Nivel de sobrepeso = Normal / Diabetes=si) = $\frac{0/9}{4/9}$ = 0
- p(Nivel de sobrepeso = Sobrepeso / Diabetes=si) = $\frac{2/9}{4/9}$ = 1/2
- p(Nivel de sobrepeso = Obeso / Diabetes=si) = $\frac{2/9}{4/9}$ = 1/2


- p(Nivel de sobrepeso = Normal / Diabetes=no) = $\frac{3/9}{5/9}$ = 3/5
- p(Nivel de sobrepeso = Sobrepeso / Diabetes=no) = $\frac{1/9}{5/9}$ = 1/5
- p(Nivel de sobrepeso = Obeso / Diabetes=no) = $\frac{1/9}{5/9}$ = 1/5

Ademas:

- $\sigma_{Glucosa\_no} = 15.8$ y $\bar{Glucosa\_no} = 90$  
- $\sigma_{Glucosa\_si} = 12.9$ y $\bar{Glucosa\_si} = 135$

Luego, se tiene constituido el clasificador tipo:

$$ Clase_{NBC} = arg max \{ p(c_i) \Pi^p_i p(a_j / c_i)) \} $$

Si llega un individuo con **Glucosa=90 mg/mL** y su **Nivel de sobrepeso = Obeso**, se tiene:

$$ Clase_{NBC} = arg max \{ \\ 
4/9 \cdot (\text{p(NS = Obeso / Diabetes=si)} \cdot \text{p(NG = 90 / Diabetes=si}), \\ 5/9 \cdot (\text{p(NS = Obeso / Diabetes=no)} \cdot \text{p(NG = 90 / Diabetes=no}) \\ \} $$


Quedando:

$$ Clase_{NBC} = arg max \{ 0.0000155 , 0.0028 \} $$

Por lo tanto el nuevo paciente no tiene diabetes segun el clasificador bayesiano.

In [9]:
library("e1071")
library("caret")

Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


In [6]:
example = data.frame(
    ns=c("Normal", "Normal", "Sobrepeso", "Normal", "Obeso", "Sobrepeso", "Sobrepeso", "Obeso", "Obeso"),
    ng=c(70, 80, 90, 100, 110, 120, 130, 140, 150),
    d=c("no","no",0,0,0,1,1,1,1)
)

In [7]:
example

ns,ng,d
Normal,70,0
Normal,80,0
Sobrepeso,90,0
Normal,100,0
Obeso,110,0
Sobrepeso,120,1
Sobrepeso,130,1
Obeso,140,1
Obeso,150,1


In [23]:
bayesian.classifier = naiveBayes(d ~ ., data = example)
bayesian.classifier


Naive Bayes Classifier for Discrete Predictors

Call:
naiveBayes.default(x = X, y = Y, laplace = laplace)

A-priori probabilities:
Y
        0         1 
0.5555556 0.4444444 

Conditional probabilities:
   ns
Y   Normal Obeso Sobrepeso
  0    0.6   0.2       0.2
  1    0.0   0.5       0.5

   ng
Y   [,1]     [,2]
  0   90 15.81139
  1  135 12.90994


In [31]:
pacients = data.frame(ns=c("Obeso", "Obeso"), ng=c(90, 100))
response = predict(object = bayesian.classifier, example)

In [32]:
response

factor(0)
Levels:

In [16]:
columns = c("area", "perimeter", "compactness", "length", "width", "AC", "lengthGroove", "class")
url = "https://www.dl.dropboxusercontent.com/s/wrexlo5im3g5ioi/seeds_dataset.csv"
seeds = read.csv(url, header = F, sep=",", col.names = columns)
seeds$class = factor(seeds$class, levels = c(1,2,3), labels = c("Kama", "Rosa", "Canadian"))

In [17]:
training.index = createDataPartition(seeds$class, p=0.7)$Resample1
training.set = seeds[training.index, ]
test.set = seeds[-training.index, ]

In [18]:
bayesian.classifier = naiveBayes(class ~ ., data = training.set)
bayesian.classifier


Naive Bayes Classifier for Discrete Predictors

Call:
naiveBayes.default(x = X, y = Y, laplace = laplace)

A-priori probabilities:
Y
     Kama      Rosa  Canadian 
0.3333333 0.3333333 0.3333333 

Conditional probabilities:
          area
Y              [,1]      [,2]
  Kama     14.41918 1.0881671
  Rosa     18.35367 1.3939059
  Canadian 11.95694 0.6507502

          perimeter
Y              [,1]      [,2]
  Kama     14.33388 0.4967680
  Rosa     16.15184 0.6021582
  Canadian 13.27755 0.3045183

          compactness
Y               [,1]       [,2]
  Kama     0.8807143 0.01746446
  Rosa     0.8827959 0.01501460
  Canadian 0.8517143 0.02191883

          length
Y              [,1]      [,2]
  Kama     5.521041 0.2152159
  Rosa     6.164490 0.2538193
  Canadian 5.227755 0.1358192

          width
Y              [,1]      [,2]
  Kama     3.262286 0.1704066
  Rosa     3.673551 0.1802931
  Canadian 2.869551 0.1473222

          AC
Y              [,1]     [,2]
  Kama     2.618596 1.269079
  

In [21]:
bayesian.results = predict(object = bayesian.classifier, test.set)
bayesian.results

[1] Kama     Kama     Kama     Kama     Kama     Kama     Canadian Kama    
 [9] Kama     Kama     Kama     Kama     Rosa     Kama     Kama     Kama    
[17] Kama     Kama     Canadian Canadian Kama     Rosa     Rosa     Rosa    
[25] Rosa     Rosa     Rosa     Rosa     Rosa     Rosa     Rosa     Rosa    
[33] Rosa     Rosa     Rosa     Rosa     Rosa     Rosa     Kama     Rosa    
[41] Rosa     Rosa     Canadian Canadian Canadian Canadian Canadian Canadian
[49] Canadian Canadian Canadian Canadian Canadian Canadian Canadian Canadian
[57] Canadian Canadian Canadian Canadian Kama     Canadian Canadian
Levels: Kama Rosa Canadian

In [20]:
conf.matrix.cb = confusionMatrix(table(test.set$class, bayesian.results))
print(conf.matrix.cb)

Confusion Matrix and Statistics

          bayesian.results
           Kama Rosa Canadian
  Kama       17    1        3
  Rosa        1   20        0
  Canadian    1    0       20

Overall Statistics
                                          
               Accuracy : 0.9048          
                 95% CI : (0.8041, 0.9642)
    No Information Rate : 0.3651          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.8571          
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: Kama Class: Rosa Class: Canadian
Sensitivity               0.8947      0.9524          0.8696
Specificity               0.9091      0.9762          0.9750
Pos Pred Value            0.8095      0.9524          0.9524
Neg Pred Value            0.9524      0.9762          0.9286
Prevalence                0.3016      0.3333          0.3651
Detection Rate          